In [1]:
import pandas as pd

def assign_trip_code(df_single_ap):
    df_single_ap['trip_code']=None
    df_single_ap = df_single_ap.reset_index(drop=True)
    date_old = 0
    trip_count_old = 0
    trip_code = 0
    
    for idx,row in df_single_ap.iterrows():
        #print(idx)
        cur_trip_count = df_single_ap.iloc[idx].trip_prevs
        cur_date = df_single_ap.iloc[idx].date
        #print(cur_trip_count, trip_count_old, trip_code)
        
        if (trip_count_old ) == cur_trip_count:
            df_single_ap.at[idx,'trip_code'] = str(trip_code)
        elif(trip_count_old +1 ) == cur_trip_count:
            #if date_old == cur_date:
            df_single_ap.at[idx,'trip_code'] =  str(trip_code)
        else:
            trip_code += 1
            df_single_ap.at[idx,'trip_code'] =  str(trip_code)

        trip_count_old = cur_trip_count
        date_old = cur_date

    return df_single_ap 


In [2]:
csv='data/2019-04_2019-08_GyokuSendo.csv'
final_csv='data/2019-04_2019-08_GyokuSendo_staypoints.csv'

df = pd.read_csv(csv,usecols=['serial','tripid','tripcount','tlm_datagettime','lat','lon'])
df.rename(columns = {'serial':'ap_id','tlm_datagettime':'timestamp'}, inplace = True)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df=df.sort_values(by=['timestamp'])

In [3]:
#df =df.query("ap_id=='AR710908'")


In [4]:
arr_ap_ids = df.ap_id.unique()
#ap_id = arr_ap_ids[0]
#ap_id

arr_trip_summary = []

for ap_id in arr_ap_ids:
    df_single_ap = df.query("ap_id=='"+ap_id+"'").copy()
    df_single_ap = df_single_ap.sort_values(by=['timestamp'])
    arr_trips = df_single_ap.tripcount.unique()
    #trip_count = arr_trips[0]
    #trip_count
    ts_prevs = df_single_ap.timestamp.min()
    lat_prevs = df_single_ap.iloc[0].lat
    lon_prevs = df_single_ap.iloc[0].lon
    
    trip_count_prevs = -9
    for trip_count in arr_trips:
        
        '''
        if (trip_count -1) != trip_count_prevs: # do not process trips which are not consicutive  e.g. trip_count_prevs = 10 and  trip_count= 13
            trip_count_prevs = trip_count
            print(trip_count)
            continue
        '''
         
        # process consicutive trips only e.g. trip_count_prevs = 12 and  trip_count=13
        df_single_trip = df_single_ap.query("tripcount=='"+str(trip_count)+"'").copy()
        df_single_trip = df_single_trip.sort_values(by=['timestamp'])
        max_ts = df_single_trip.timestamp.max()
        min_ts= df_single_trip.timestamp.min()
        
        stay_time =  min_ts - ts_prevs
        trip_time = (max_ts - min_ts).total_seconds()
        
        #if (trip_time < 1800): # ignore if the stay_time is less than 30 minutes
        #    continue
          
        
        lat_min = df_single_trip.iloc[0].lat
        lon_min = df_single_trip.iloc[0].lon
        lat_max = df_single_trip.iloc[len(df_single_trip)-1].lat 
        lon_max = df_single_trip.iloc[len(df_single_trip)-1].lon
        
        avg_lat = (lat_prevs + lat_min)/2
        avg_lon = (lon_prevs + lon_min)/2
        
        lat_prevs = lat_max
        lon_prevs = lon_max
        
        if ( trip_count_prevs  == trip_count-1):
            
            arr_trip_summary.append({
                'ap_id': ap_id,

                'trip_prevs': trip_count_prevs,
                'trip_count': trip_count,
                'ts_prevs': ts_prevs,
                'ts_min': min_ts,
                #'ts_max': max_ts,#
                #'trip_time': trip_time,#
                'stay_time': stay_time.total_seconds(),
                'avg_lon_with_prvs': avg_lon,
                'avg_lat_with_prvs': avg_lat,
                #'lat_min': lat_min,#
                #'lon_min': lon_min,#
                #'lat_max': lat_max,    #        
                #'lon_max': lon_max #
                })
            
        trip_count_prevs = trip_count
        ts_prevs = max_ts
    
trip_df =  pd.DataFrame(arr_trip_summary)      
trip_df['date'] = trip_df['ts_min'].dt.date

# remove entries with stay_time < 1800 seconds
trip_df = trip_df[trip_df['stay_time']>=1800]# 

In [5]:
trip_df.query("ap_id=='AR710908'")

,ap_id,trip_prevs,trip_count,ts_prevs,ts_min,stay_time,avg_lon_with_prvs,avg_lat_with_prvs,date
0,AP521745,2080,2081,2019-04-01 08:49:43,2019-04-01 11:14:20,8677.0,127.748598,26.141213,2019-04-01
1,AP521745,2101,2102,2019-04-04 12:15:45,2019-04-04 18:05:37,20992.0,127.747382,26.141103,2019-04-04
2,AP521745,2123,2124,2019-04-08 11:23:41,2019-04-08 13:51:25,8864.0,127.749557,26.141103,2019-04-08
3,AP521745,2152,2153,2019-04-13 10:59:40,2019-04-13 12:58:27,7127.0,127.750141,26.140792,2019-04-13
4,AP521745,2269,2270,2019-04-25 10:32:25,2019-04-25 14:24:14,13909.0,127.748954,26.141355,2019-04-25


In [6]:
len(trip_df)

4935

# merge consicutive trips on same day

In [12]:
arr_ap_ids = trip_df.ap_id.unique()

arr_trip_merged = []
for ap_id in arr_ap_ids:
    df_single_ap = trip_df.query("ap_id=='"+ap_id+"'").copy()
    #display(df_single_ap)
    df_single_ap = assign_trip_code(df_single_ap)
    if ap_id == 'AR710908':
        display(df_single_ap)
    
    arr_trip_code = df_single_ap.trip_code.unique()

    for trip_code in arr_trip_code:
        df_trip_code = df_single_ap.query("trip_code=='"+str(trip_code)+"'")
        
        arr_trip_merged.append( {
                    'ap_id': ap_id,
                    'date':df_trip_code.date.min(),
                    'trip_prevs': df_trip_code.trip_prevs.min(),
                    'trip_count': df_trip_code.trip_count.max(),
                    'ts_prevs': df_trip_code.ts_prevs.min(),
                    'ts_car_start': df_trip_code.ts_min.max(),
                    'stay_time': df_trip_code.stay_time.sum(),
                    'avg_lon_with_prvs': df_trip_code.avg_lon_with_prvs.mean(),
                    'avg_lat_with_prvs':  df_trip_code.avg_lat_with_prvs.mean(),
                    } )

            
df_final =  pd.DataFrame(arr_trip_merged)      
df_final.to_csv(final_csv, index=False) 

,ap_id,trip_prevs,trip_count,ts_prevs,ts_min,stay_time,avg_lon_with_prvs,avg_lat_with_prvs,date,trip_code
0,AR710908,188,189,2019-08-13 10:20:27,2019-08-13 12:11:37,6670.0,127.749944,26.141371,2019-08-13,1
1,AR710908,333,334,2019-08-30 14:19:31,2019-08-30 16:50:52,9081.0,127.748111,26.141853,2019-08-30,2


In [13]:
len(df_final)

4846

In [14]:
df_final.head()

,ap_id,date,trip_prevs,trip_count,ts_prevs,ts_car_start,stay_time,avg_lon_with_prvs,avg_lat_with_prvs
0,AP521745,2019-04-01,2080,2081,2019-04-01 08:49:43,2019-04-01 11:14:20,8677.0,127.748598,26.141213
1,AP521745,2019-04-04,2101,2102,2019-04-04 12:15:45,2019-04-04 18:05:37,20992.0,127.747382,26.141103
2,AP521745,2019-04-08,2123,2124,2019-04-08 11:23:41,2019-04-08 13:51:25,8864.0,127.749557,26.141103
3,AP521745,2019-04-13,2152,2153,2019-04-13 10:59:40,2019-04-13 12:58:27,7127.0,127.750141,26.140792
4,AP521745,2019-04-25,2269,2270,2019-04-25 10:32:25,2019-04-25 14:24:14,13909.0,127.748954,26.141355


In [15]:
trip_df1 = df_final.query("ap_id=='AR710908'")
trip_df1.head()

,ap_id,date,trip_prevs,trip_count,ts_prevs,ts_car_start,stay_time,avg_lon_with_prvs,avg_lat_with_prvs
4514,AR710908,2019-08-13,188,189,2019-08-13 10:20:27,2019-08-13 12:11:37,6670.0,127.749944,26.141371
4515,AR710908,2019-08-30,333,334,2019-08-30 14:19:31,2019-08-30 16:50:52,9081.0,127.748111,26.141853
